In [1]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict

In [ ]:
## Uun this code when using it for the first time for loading the dataset
# pip install huggingface_hub

In [2]:
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv")

C:\Users\liu.xuanc\AppData\Local\Temp\ipykernel_11624\2761146993.py:1: DtypeWarning: Columns (2,3,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv")


First step analysis of chords

In [3]:
df['chords']

0         <intro_1> C <verse_1> F C E7 Amin C F C G7 C F...
1         <intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs...
2         <intro_1> Csmin <verse_1> A Csmin A Csmin A Cs...
3         <intro_1> D Dmaj7 D Dmaj7 <verse_1> Emin A D G...
4         <intro_1> C <verse_1> G C G C <chorus_1> F Dmi...
                                ...                        
679802    D G D A G D A D G D A G D A D G D A G D A D G ...
679803    G Gadd13 G7 Gadd13 G Emin A7 Emin A7 Emin A7 E...
679804    E Fs E Fs E Fs E Fs E Fs E Fs B Cs Fs B Cs Fs ...
679805    E Csmin Fsmin B E Csmin Fsmin B E Csmin Fsmin ...
679806    A B7 E7 A Fs7 A E7 A D A D B7 A B7 E7 A Fs7 A ...
Name: chords, Length: 679807, dtype: object

In [4]:
NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# A simple homophone conversion dictionary (for standardized input)
FLAT_TO_SHARP = {
    'Db': 'C#', 'Eb': 'D#', 'Gb': 'F#', 'Ab': 'G#', 'Bb': 'A#',
    'db': 'C#', 'eb': 'D#', 'gb': 'F#', 'ab': 'G#', 'bb': 'A#'
}

In [8]:
def analyze_pandas_series(series_data, top_n=50):
    print("Removing labels...")
    clean_series = series_data.str.replace(r'<[^>]+>', ' ', regex=True)

    print("Expliciting and counting...")
    # 1. .str.split(): to list ['C', 'F', 'G']
    # 2. .explode(): turns lists to one list
    # 3. .value_counts(): count
    chord_counts = clean_series.str.split().explode().value_counts()
    
    print(f"=== Finish ===")
    print(f"Chords amount: {chord_counts.sum()}")
    print(f"Chords type amount: {len(chord_counts)}")
    print("-" * 30)
    
    print(f"Top {top_n} commonly used chords:")
    print(chord_counts.head(top_n))

    print("-" * 30)
    print("Examples of long tail data (with fewer occurrences):")
    print(chord_counts.tail(20).index.tolist())
    
    return chord_counts

In [6]:
def simplify_slash_chord(text):
    if not isinstance(text, str): return ""
    
    chords = text.split()
    simplified = [c.split('/')[0] for c in chords]
    return " ".join(simplified)

print("Dealing with slash chords...")
df['chords_simplified'] = df['chords'].apply(simplify_slash_chord)
print("Finish!")

Dealing with slash chords...


In [9]:
# Analyze simplified chords
counts = analyze_pandas_series(df['chords_simplified'])

Removing labels...
Expliciting and counting...
=== Finish ===
Chords amount: 51994634
Chords type amount: 749
------------------------------
Top 50 commonly used chords:
chords_simplified
G        7161058
C        5945092
D        5413274
A        3952888
F        3389952
Amin     2958979
E        2719857
Emin     2662799
Bmin     1446212
B        1384402
Dmin     1265370
Bb       1033272
Fsmin     887691
Fs        692455
Csmin     613688
Gmin      556347
Eb        508770
Cmin      409476
Ab        365022
Cs        328877
Gsmin     324517
A7        302780
Fmin      294103
D7        288826
Amin7     287833
Emin7     284727
E7        271749
Gs        267858
B7        253029
G7        248206
Cadd9     235057
Db        194678
As        166455
Bmin7     164123
Fmaj7     162679
Dmin7     158330
Cmaj7     155171
Ds        148442
C7        147180
Bbmin     141925
Dsmin     115335
Ano3d     105842
Dsus4     103025
Dno3d     102338
Gb         99587
Gno3d      98220
Eno3d      90866
Dsus2      89

In [10]:
## Check the long tail data
total_count = counts.sum()
top_50_count = counts.head(50).sum()

top_50_chords = set(counts.head(50).index)

# 5% long tail data
tail_chords = [c for c in counts.index if c not in top_50_chords]

print(f"Top 50 coverage ratie: {top_50_count / total_count:.4f}")
print(f"Number of long tail chords type: {len(tail_chords)}")

print("\n Long tail data example (Rank 51-70):")
print(counts.iloc[50:70])

print("\n Rare chords example:")
print(counts.tail(20))

Top 50 coverage ratie: 0.9458
Number of long tail chords type: 699

 Long tail data example (Rank 51-70):
chords_simplified
Fsmin7    83258
Asus2     80833
Gmin7     79634
Gmaj7     79039
Ebmin     78919
Csmin7    72221
Cno3d     72148
Bno3d     71021
F7        66986
Asmin     64547
Amaj7     60432
Gadd13    59584
Fs7       59092
Cmin7     58856
Gsus4     46604
Esus4     46589
Fno3d     44630
Abmin     39087
Fmin7     38177
Fsno3d    36772
Name: count, dtype: int64

 Rare chords example:
chords_simplified
Cs13b9        1
Edimb7        1
E11s          1
Dbmaj11       1
Bminmaj9      1
Gs13b9        1
sC            1
Fsminmaj13    1
Fsaugmaj11    1
Bbdim13b9     1
Eminmaj13     1
A11s          1
Abdim9        1
Eb11b9        1
Fsdim11       1
Bbdimadd13    1
Gdim11b9      1
Dsdim9        1
Bdim11b9      1
Fminmaj13     1
Name: count, dtype: int64


Clean chords data:
1. Seperate one song into several segments;
2. Infer the key of each segment;
3. Transpose chords into Roman numerals.

In [11]:
# ==========================================
# Global configuration and auxiliary functions
# ==========================================

def parse_chord_root_quality(chord_str):
    """
    Auxiliary function: parses a single chord string and returns (Root, Quality)
    For example: "Am" ->("A", "min"), "G7" ->("G", "7")
    """
    chord_str = chord_str.strip()
    if not chord_str: return None, None
    
    for flat, sharp in FLAT_TO_SHARP.items():
        if chord_str.startswith(flat):
            chord_str = sharp + chord_str[len(flat):]
            break
    if len(chord_str) > 1 and chord_str[1] == 's': # Fs7
         chord_str = chord_str[0] + '#' + chord_str[2:]
         
    match = re.match(r'([A-G]#?)(.*)', chord_str)
    if not match: return None, None
    
    root = match.group(1)
    rest = match.group(2).lower()
    
    # Characteristic
    if 'dim' in rest: quality = 'dim'
    elif 'min' in rest or 'm' == rest: quality = 'min'
    elif '7' in rest and 'maj' not in rest and 'min' not in rest: quality = '7'
    else: quality = 'maj' # Default to major triad/major seventh chord
    
    return root, quality

In [12]:
# ==========================================
# Segmentation
# ==========================================

def segment_music_string(raw_string):
    """
    Input: '<intro_1> C D <verse_1> F G'
    Output: [{'label': 'intro_1', 'chords': ['C', 'D']}, {'label': 'verse_1', 'chords': ['F', 'G']}]
    """
    parts = re.split(r'(<[^>]+>)', raw_string)
    
    segments = []
    current_label = "Unknown"
    
    for part in parts:
        part = part.strip()
        if not part: continue
        
        # Label (< xxx，> )
        if part.startswith('<') and part.endswith('>'):
            current_label = part.strip('<>')
        else:
            # Content（Chords）
            chord_list = part.split()
            if chord_list:
                segments.append({
                    'label': current_label,
                    'chords': chord_list
                })
                
    return segments

In [13]:
# ==========================================
# Key Inference
# ==========================================

KEY_TEMPLATES = {}
def _build_templates():
    for i in range(12):
        root = NOTES[i]
        # Major key template
        s_maj = [NOTES[(i + n) % 12] for n in [0, 2, 4, 5, 7, 9, 11]]
        maj_chords = {s_maj[0]:['maj'], s_maj[1]:['min'], s_maj[2]:['min'], s_maj[3]:['maj'], s_maj[4]:['maj','7'], s_maj[5]:['min'], s_maj[6]:['dim']}
        KEY_TEMPLATES[f"{root} Major"] = (maj_chords, s_maj[0], s_maj[4]) # (Template, main tone, subordinate tone)
        
        # Minor Tune Template (Natural+Harmony)
        s_min = [NOTES[(i + n) % 12] for n in [0, 2, 3, 5, 7, 8, 10]]
        dom_root = NOTES[(i + 7) % 12]
        min_chords = {s_min[0]:['min'], s_min[1]:['dim'], s_min[2]:['maj'], s_min[3]:['min'], s_min[4]:['min'], dom_root:['maj','7'], s_min[5]:['maj'], s_min[6]:['maj','7']}
        KEY_TEMPLATES[f"{root} Minor"] = (min_chords, s_min[0], dom_root)
_build_templates()

def infer_key_from_list(chord_list):
    """
    input: ['C', 'F', 'G7', 'C']
    output: 'C Major'
    """
    if not chord_list: return "Unknown"
    
    parsed_data = []
    for idx, c_str in enumerate(chord_list):
        r, q = parse_chord_root_quality(c_str)
        if r:
            next_r = None
            if idx + 1 < len(chord_list):
                next_r, _ = parse_chord_root_quality(chord_list[idx+1])
            parsed_data.append((r, q, next_r))
            
    scores = defaultdict(int)
    
    # calculate the score for each
    for key_name, (template, tonic, dom) in KEY_TEMPLATES.items():
        score = 0
        for root, quality, next_root in parsed_data:
            # Basic mathc
            if root in template:
                score += 1
                if quality in template[root]:
                    score += 2
            else:
                score -= 1
            
            if root == dom and quality == '7' and next_root == tonic:
                score += 5
                
            if root == tonic:
                score += 1
                
        scores[key_name] = score
        
    if not scores: return "Unknown"
    return max(scores, key=scores.get)

In [20]:
# ==========================================
# Transpose the chords to Roman numerals
# ==========================================

ROMAN_MAP = {
    0: 'I', 1: 'bII', 2: 'II', 3: 'bIII', 4: 'III', 5: 'IV', 
    6: 'bV', 7: 'V', 8: 'bVI', 9: 'VI', 10: 'bVII', 11: 'VII'
}

def convert_to_roman(chord_list, key_str):
    """
    input: chord_list=['C', 'F', 'G'], key_str='C Major'
    output: ['I', 'IV', 'V']
    """
    if key_str == "Unknown":
        return chord_list
    
    # Main tone
    key_root_str = key_str.split()[0] # "C Major" -> "C"
    is_minor_key = "Minor" in key_str
    
    if key_root_str not in NOTES: return chord_list
    key_root_idx = NOTES.index(key_root_str)
    
    roman_output = []
    
    for chord_str in chord_list:
        root, quality = parse_chord_root_quality(chord_str)
        if not root:
            roman_output.append("?")
            continue
            
        # Calculate interval  (Root note of chord - tonic) % 12
        root_idx = NOTES.index(root)
        interval = (root_idx - key_root_idx) % 12
        
        base_roman = ROMAN_MAP.get(interval, "?")
        
        # min or dim -> lowercast
        if quality == 'min' or quality == 'dim':
            final_roman = base_roman.lower()
        else:
            final_roman = base_roman
            
            
        # suffix
        if quality == '7':
            final_roman += '7'
        elif quality == 'dim':
            final_roman += '°'
            
        roman_output.append(final_roman)
        
    return roman_output

In [14]:
# ==========================================
# Fix short chords progression
# ==========================================

def patch_short_segments(analyzed_segments, min_chords=2):
    """
    Parameters:
    analyzed_segments: list, {'label':..., 'chords':..., 'key':...}
    min_chords: threshold，chords less than this quantity will be corrected (default is 2)
    """
    
    # Scan twice
    
    # orward Pass
    for i in range(1, len(analyzed_segments)):
        current_seg = analyzed_segments[i]
        prev_seg = analyzed_segments[i-1]
        
        if len(current_seg['chords']) < min_chords:
            if prev_seg['key'] != "Unknown":
                current_seg['key'] = prev_seg['key']
                current_seg['key_source'] = 'borrowed_prev' 

    # Backward Pass
    for i in range(len(analyzed_segments) - 2, -1, -1):
        current_seg = analyzed_segments[i]
        next_seg = analyzed_segments[i+1]
        
        if len(current_seg['chords']) < min_chords:
            # (Usually Intro should follow Verse's tone)
            if next_seg['key'] != "Unknown":
                 current_seg['key'] = next_seg['key']
                 current_seg['key_source'] = 'borrowed_next'
                 
    return analyzed_segments

In [21]:
# ==========================================
# Combine Roman numerals
# ==========================================

def extract_and_format_roman(analyzed_records, include_tags=True):
    """
    Parameters:
    analyzed_records: chords_output in the former step
    include_tags: 
                  - True: Keep "<intro> I IV <verse> I V" (Structure + Sequence)
                  - False: Only "I IV I V" (Sequence)
    
    Return:
    str: Roman numerals string
    """
    output_parts = []
    
    for record in analyzed_records:
        roman_seq = " ".join(record['roman'])
        
        if include_tags:
            # "<label> roman_seq"
            segment_str = f"<{record['section']}> {roman_seq}"
            output_parts.append(segment_str)
        else:
            # Roman numerals
            output_parts.append(roman_seq)
            
    return " ".join(output_parts)

In [16]:
df['chords'][0]

'<intro_1> C <verse_1> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <verse_2> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <chorus_1> F C F C G C F C E7 Amin C F G7 C <solo_1> D <chorus_2> G D G D A D G D Fs7 Bmin D G A7 D G A7 D'

In [17]:
# Segmentation
segments = segment_music_string(df['chords'][0])
segments

[{'label': 'intro_1', 'chords': ['C']},
 {'label': 'verse_1',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'verse_2',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'chorus_1',
  'chords': ['F',
   'C',
   'F',
   'C',
   'G',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'solo_1', 'chords': ['D']},
 {'label': 'chorus_2',
  'chords': ['G',
   'D',
   'G',
   'D',
   'A',
   'D',
   'G',
   'D',
   'Fs7',
   'Bmin',
   'D',
   'G',
   'A7',
   'D',
   'G',
   'A7',
   'D']}]

In [18]:
# Infer the key
analyzed_records = []
for seg in segments:
    inferred_key = infer_key_from_list(seg['chords'])
    analyzed_records.append({
        'section': seg['label'],
        'chords': seg['chords'],
        'key': inferred_key
    })

analyzed_records

[{'section': 'intro_1', 'chords': ['C'], 'key': 'C Major'},
 {'section': 'verse_1',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'section': 'verse_2',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'section': 'chorus_1',
  'chords': ['F',
   'C',
   'F',
   'C',
   'G',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'section': 'solo_1', 'chords': ['D'], 'key': 'D Major'},
 {'section': 'chorus_2',
  'chords': ['G',
   'D',
   'G',
   'D',
   'A',
   'D',
   'G',
   'D',
   'Fs7',
   'Bmin',
   'D',
   'G',
   'A7',
   'D',
   'G',
   'A7',
   'D'],
  'key': 'D Major'}]

In [25]:
# Fix short patch
analyzed_records = patch_short_segments(analyzed_records, min_chords=2)

# Transpost to Roman numerals
for record in analyzed_records:
    record['roman'] = convert_to_roman(record['chords'], record['key'])

In [26]:
record

{'section': 'chorus_2',
 'chords': ['G',
  'D',
  'G',
  'D',
  'A',
  'D',
  'G',
  'D',
  'Fs7',
  'Bmin',
  'D',
  'G',
  'A7',
  'D',
  'G',
  'A7',
  'D'],
 'key': 'D Major',
 'roman': ['IV',
  'I',
  'IV',
  'I',
  'V',
  'I',
  'IV',
  'I',
  'III7',
  'vi',
  'I',
  'IV',
  'V7',
  'I',
  'IV',
  'V7',
  'I']}

In [27]:
roman_numerals_record = extract_and_format_roman(analyzed_records, include_tags=False)
roman_numerals_record

'I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I IV I V I IV I III7 vi I IV V7 I I IV I IV I V I IV I III7 vi I IV V7 I IV V7 I'

Coarse graining for Roman Numerals

In [28]:
class RomanNormalizer:
    def __init__(self, strategy='core'):
        """
        strategy:
          - 'core': Keep only I, i, V7, dim
          - 'standard': Add sus, maj7, m7
          - 'dynamic': Keep all details
        """
        self.strategy = strategy

    def normalize(self, raw_roman):
        if not isinstance(raw_roman, str) or raw_roman in ['?', 'Unknown', '<ERROR>']:
            return None

        # Disassemble (prefix core suffix)
        match = re.match(r'^([b#]*)([ivIV]+)(.*)$', raw_roman)
        if not match: return None
        
        prefix, core, suffix = match.groups()
        is_lower = core.islower()

        # Reorganize strings based on strategy
        if self.strategy == 'dynamic':
            return raw_roman
            
        elif self.strategy == 'standard':
            new_suffix = ""
            if 'sus4' in suffix: new_suffix = 'sus4'
            elif 'sus2' in suffix: new_suffix = 'sus2'
            elif 'maj7' in suffix or 'M7' in suffix: new_suffix = 'maj7'
            elif 'ø' in suffix: new_suffix = 'm7b5'
            elif 'dim7' in suffix: new_suffix = 'dim7'
            elif 'dim' in suffix or '°' in suffix: new_suffix = 'dim'
            elif '7' in suffix: new_suffix = '7' # 保留 V7 或 m7
            return f"{prefix}{core}{new_suffix}"

        elif self.strategy == 'core':
            new_suffix = ""
            # Two situations keeping suffix:
            # 1. dominant 7th
            if not is_lower and '7' in suffix and 'maj' not in suffix:
                new_suffix = '7'
            # 2. diminish
            elif 'dim' in suffix or '°' in suffix:
                new_suffix = 'dim'
            # others
            return f"{prefix}{core}{new_suffix}"
            
        return raw_roman

In [29]:
def normalize_sequence(roman_seq_str, normalizer):
    """
    input: "<intro> I bVIsus4 V7"
    output: "<intro> I bVI V7" (core strategy)
    """
    if not isinstance(roman_seq_str, str):
        return ""
    
    tokens = roman_seq_str.split()
    
    normalized_tokens = []
    for token in tokens:
        # if label (<intro>), keep it
        if token.startswith('<') and token.endswith('>'):
            normalized_tokens.append(token)
        else:
            norm = normalizer.normalize(token)
            if norm:
                normalized_tokens.append(norm)
    
    # append to a string list
    return " ".join(normalized_tokens)



In [30]:
normalizer = RomanNormalizer(strategy='core')

normalized_roman = normalize_sequence(roman_numerals_record, normalizer)
print(normalized_roman) 

I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I IV I V I IV I III7 vi I IV V7 I I IV I IV I V I IV I III7 vi I IV V7 I IV V7 I


Tool: Roman Numerals to scales (Used as a dictionary)

In [31]:
def map_roman_to_pitches(roman_str):
    """
    Roman numerals to music scales [0-11].
    """
    if not roman_str: return []

    ROMAN_OFFSETS = {'i': 0, 'ii': 2, 'iii': 4, 'iv': 5, 'v': 7, 'vi': 9, 'vii': 11}
    
    INTERVALS = {
        'maj': [0, 4, 7], 'min': [0, 3, 7], 'dim': [0, 3, 6],
        'dom7': [0, 4, 7, 10], 'maj7': [0, 4, 7, 11], 'min7': [0, 3, 7, 10],
        'dim7': [0, 3, 6, 9], 'm7b5': [0, 3, 6, 10],
        'sus4': [0, 5, 7], 'sus2': [0, 2, 7]
    }
    
    EXTENSIONS = {'9': 2, '11': 5, '13': 9, 'b9': 1, '#9': 3}

    # Analyze
    match = re.match(r'^([b#]*)([ivIV]+)(.*)$', roman_str)
    if not match: return []
    prefix, core, suffix = match.groups()

    # Root
    base = ROMAN_OFFSETS.get(core.lower())
    if base is None: return []
    acc = -1 if prefix=='b' else (1 if prefix=='#' else 0)
    root = (base + acc) % 12

    # Basic interval
    is_lower = core.islower()

    intervals = []
    if 'sus4' in suffix: intervals = INTERVALS['sus4']
    elif 'sus2' in suffix: intervals = INTERVALS['sus2']
    elif 'maj7' in suffix: intervals = INTERVALS['maj7']
    elif 'dim7' in suffix: intervals = INTERVALS['dim7']
    elif 'dim' in suffix: intervals = INTERVALS['dim']
    elif '7' in suffix: intervals = INTERVALS['min7'] if is_lower else INTERVALS['dom7']
    else: intervals = INTERVALS['min'] if is_lower else INTERVALS['maj']
    
    current_intervals = list(intervals)

    # Deal with extentions
    for ext, val in EXTENSIONS.items():
        if ext in suffix and val not in current_intervals:
            current_intervals.append(val)

    # (Root + Interval) % 12
    pitches = sorted(list(set([(root + i) % 12 for i in current_intervals])))
    
    return pitches